In [174]:
import pandas as pd
import numpy as np
import csv
import math
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [175]:
price = pd.read_excel('multi_asset_etf_data.xlsx',sheet_name='prices', index_col=0)
TR=pd.read_excel('multi_asset_etf_data.xlsx',sheet_name='total returns', index_col=0)
ER=pd.read_excel('multi_asset_etf_data.xlsx',sheet_name='excess returns', index_col=0)

In [176]:
ER.head()

,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SPY,TIP
Date,,,,,,,,,,,
2009-04-30,0.008441,-0.001553,0.155030,0.114637,0.137907,-0.028004,0.295598,0.229650,0.022329,0.098793,-0.018505
2009-05-31,0.054143,0.163134,0.159871,0.132389,0.029026,-0.020303,0.023198,0.054363,0.028336,0.058925,0.020437
2009-06-30,0.004550,-0.026858,-0.023094,-0.014648,0.032919,-0.006170,-0.025462,0.041443,-0.004035,-0.001254,0.001383
2009-07-31,0.031312,0.018595,0.110173,0.100442,0.069217,0.008344,0.105826,0.143274,0.015353,0.074633,0.000906
2009-08-31,0.007193,-0.040800,-0.013571,0.044595,-0.017404,0.007199,0.131504,0.032977,-0.004586,0.036504,0.007979


In [193]:
mean = np.array(ER.mean(axis=0)*12)
vol=np.array(ER.std()*math.sqrt(12))
SP=mean/vol
print('Mean\n',mean)
print('\nVolatility\n',vol)
print('\nSharpe Ratio\n',SP)

Mean
 [-0.00184326  0.02544345  0.06488728  0.08159688  0.06416829  0.01426904
  0.12947332  0.07993823  0.01897443  0.14372749  0.03672076]

Volatility
 [0.08335856 0.17897535 0.19653122 0.16599122 0.08915408 0.06240508
 0.18710106 0.22738651 0.05080985 0.14767864 0.05152929]

Sharpe Ratio
 [-0.0221124   0.14216176  0.3301627   0.49157349  0.71974602  0.22865192
  0.69199673  0.35155222  0.37343997  0.97324498  0.71261912]


In [180]:
print('The asset with largest Sharpe ratio is', ER.columns[np.argmax(SP)])
print('The asset with worst Sharpe ratio is', ER.columns[np.argmin(SP)])

The asset with largest Sharpe ratio is SPY
The asset with worst Sharpe ratio is BWX


In [184]:
corr = ER.corr()
cov=ER.cov()
stacked_corr = corr.stack()
highest_corr_pairs = stacked_corr.nlargest(12)
lowest_corr_pairs = stacked_corr.nsmallest(5)
print('Highest correlation pair is ',highest_corr_pairs.index[11])
print('Lowest correlation pair is ',lowest_corr_pairs.index[0])

Highest correlation pair is  ('PSP', 'SPY')
Lowest correlation pair is  ('DBC', 'IEF')


In [186]:
#calculate W_Tan based on fomulas
W_tan=(1/(np.transpose(np.ones(11))@np.linalg.inv(cov)@mean))*np.linalg.inv(cov)@mean
print('W_tan=',W_tan)

W_tan= [-1.46497414  0.02843646  0.26102769  0.45291399  1.52894242  1.89399165
 -0.24277199 -1.2710546  -3.13344544  2.58999871  0.35693524]


In [187]:
print('The W_tan aligns perfectly with Sharpe ratio of each asset')

The W_tan aligns perfectly with Sharpe ratio of each asset


In [194]:
mean_W_tan=np.transpose(W_tan)@mean
vol_W_tan=math.sqrt(np.transpose(W_tan)@cov@W_tan)
SP_W_tan=mean_W_tan/vol_W_tan
print('Mean of W_tan=',mean_W_tan)
print('Volatility of W_tan=',vol_W_tan)
print('Sharpe Ratio of W_tan=',SP_W_tan)

Mean of W_tan= 0.37531992061454567
Volatility of W_tan= 0.05528789276868884
Sharpe Ratio of W_tan= 6.788464920969106


In [195]:
#We drop Tip in ER
ER1=ER.drop('TIP', axis=1)
W_tan1=(1/(np.transpose(np.ones(10))@np.linalg.inv(ER1.cov()*math.sqrt(12))@ER1.mean()*12))*np.linalg.inv(ER1.cov()*math.sqrt(12))@ER1.mean()*12
print(W_tan1)
mean_W_tan1=np.transpose(W_tan1)@ER1.mean()*12
vol_W_tan1=math.sqrt(np.transpose(W_tan1)@ER1.cov()*math.sqrt(12)@W_tan1)
SP_W_tan1=mean_W_tan1/vol_W_tan1
print('Mean of W_tan without TIP=',mean_W_tan1)
print('Volatility of W_tan without TIP=',vol_W_tan1)
print('Sharpe Ratio of W_tan without TIP=',SP_W_tan1)

[-1.51274965  0.05515758  0.27808582  0.44149597  1.5931398   2.21245096
 -0.24589531 -1.31409179 -3.23895589  2.73136252]
Mean of W_tan without TIP= 0.3862908162901122
Volatility of W_tan without TIP= 0.10751662610569362
Sharpe Ratio of W_tan without TIP= 3.592847267271679


In [196]:
ER['TIP']+=0.0012
W_tan2=(1/(np.transpose(np.ones(11))@np.linalg.inv(ER.cov()*math.sqrt(12))@ER.mean()*12))*np.linalg.inv(ER.cov()*math.sqrt(12))@ER.mean()*12
print(W_tan2)
mean_W_tan2=np.transpose(W_tan2)@ER.mean()*12
vol_W_tan2=math.sqrt(np.transpose(W_tan2)@ER.cov()*math.sqrt(12)@W_tan2)
SP_W_tan2=mean_W_tan2/vol_W_tan2
print('Mean of W_tan adjusting TIP=',mean_W_tan2)
print('Volatility of W_tan adjusting TIP=',vol_W_tan2)
print('Sharpe Ratio of W_tan adjusting TIP=',SP_W_tan2)

[-1.12679503 -0.16070899  0.14028166  0.53373651  1.07452112 -0.36022334
 -0.22066354 -0.96641575 -2.38658944  1.5893546   2.8835022 ]
Mean of W_tan adjusting TIP= 0.3391848411984788
Volatility of W_tan adjusting TIP= 0.08082046831461605
Sharpe Ratio of W_tan adjusting TIP= 4.196769064466539
